# Facebook Prophet Stock Price Prediction

This notebook demonstrates how to use Facebook Prophet for stock price prediction with real market data.

**Features:**
- Downloads real stock data from Yahoo Finance
- Trains Prophet model with seasonality
- Evaluates model performance
- Makes future predictions with confidence intervals
- Comprehensive visualizations

**⚠️ Disclaimer:** This is for educational purposes only. Stock predictions are inherently uncertain and should not be used as the sole basis for investment decisions.

## 1. Install Required Libraries

Run this cell first to install all required packages:

In [ ]:
# Install required packages
!pip install "numpy<2" prophet yfinance pandas matplotlib plotly scikit-learn

## 2. Import Libraries

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as pyo
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

## 3. Configuration and Data Download

Configure your analysis parameters here:

In [ ]:
# Configuration - Change these parameters as needed
STOCK_SYMBOL = 'AAPL'  # Change this to any stock symbol (AAPL, GOOGL, TSLA, MSFT, etc.)
START_DATE = '2020-01-01'
END_DATE = '2024-01-01'
FORECAST_DAYS = 30  # Number of days to predict into the future

print(f"Downloading {STOCK_SYMBOL} stock data from {START_DATE} to {END_DATE}...")

# Download stock data
stock_data = yf.download(STOCK_SYMBOL, start=START_DATE, end=END_DATE)

# Display basic information about the data
print(f"Data shape: {stock_data.shape}")
print(f"Date range: {stock_data.index[0]} to {stock_data.index[-1]}")
print(f"Columns: {list(stock_data.columns)}")
print("\nFirst 5 rows:")
print(stock_data.head())

## 4. Data Preprocessing for Prophet

Prophet requires specific column names: 'ds' for dates and 'y' for values.

In [ ]:
# Reset index to make Date a column
df = stock_data.reset_index()

# Check available columns and handle different column naming
print("Available columns:", df.columns.tolist())
print("Data shape:", df.shape)

# Handle different possible column names from yfinance
if 'Adj Close' in df.columns:
    price_column = 'Adj Close'
elif ('Adj Close', STOCK_SYMBOL) in df.columns:
    price_column = ('Adj Close', STOCK_SYMBOL)
elif 'Close' in df.columns:
    price_column = 'Close'
else:
    # For multi-level columns, get the Adj Close for the specific stock
    adj_close_cols = [col for col in df.columns if 'Adj Close' in str(col)]
    if adj_close_cols:
        price_column = adj_close_cols[0]
    else:
        close_cols = [col for col in df.columns if 'Close' in str(col)]
        price_column = close_cols[0] if close_cols else df.columns[-1]

print(f"Using price column: {price_column}")

# Extract the price data and ensure it's 1-dimensional
price_data = df[price_column]

# Handle multi-dimensional data by flattening if necessary
if hasattr(price_data, 'values'):
    if len(price_data.values.shape) > 1:
        price_values = price_data.values.flatten()
    else:
        price_values = price_data.values
else:
    price_values = price_data

# Prepare data for Prophet
prophet_data = pd.DataFrame({
    'ds': df['Date'],  # Date column
    'y': price_values  # Target variable (Adjusted Close price)
})

# Remove any missing values
prophet_data = prophet_data.dropna()

# Display the prepared data
print("\nData prepared for Prophet:")
print(f"Shape: {prophet_data.shape}")
print(prophet_data.head())
print(f"\nDate range: {prophet_data['ds'].min()} to {prophet_data['ds'].max()}")
print(f"Price range: ${prophet_data['y'].min():.2f} to ${prophet_data['y'].max():.2f}")

## 5. Data Visualization

In [ ]:
# Plot the historical stock price
plt.figure(figsize=(12, 6))
plt.plot(prophet_data['ds'], prophet_data['y'], linewidth=1)
plt.title(f'{STOCK_SYMBOL} Historical Stock Price', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Adjusted Close Price ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Interactive plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=prophet_data['ds'], 
    y=prophet_data['y'],
    mode='lines',
    name=f'{STOCK_SYMBOL} Price',
    line=dict(color='blue', width=1)
))

fig.update_layout(
    title=f'{STOCK_SYMBOL} Historical Stock Price (Interactive)',
    xaxis_title='Date',
    yaxis_title='Adjusted Close Price ($)',
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

## 6. Train-Test Split

In [ ]:
# Split data into train and test sets
# We'll use 80% for training and 20% for testing
split_index = int(len(prophet_data) * 0.8)

train_data = prophet_data[:split_index].copy()
test_data = prophet_data[split_index:].copy()

print(f"Training data: {len(train_data)} rows")
print(f"Testing data: {len(test_data)} rows")
print(f"Training period: {train_data['ds'].min()} to {train_data['ds'].max()}")
print(f"Testing period: {test_data['ds'].min()} to {test_data['ds'].max()}")

## 7. Create and Train Prophet Model

In [ ]:
print("Training Prophet model...")

# Initialize Prophet model with various parameters
model = Prophet(
    daily_seasonality=True,      # Enable daily seasonality
    weekly_seasonality=True,     # Enable weekly seasonality
    yearly_seasonality=True,     # Enable yearly seasonality
    seasonality_mode='multiplicative',  # Try 'additive' or 'multiplicative'
    changepoint_prior_scale=0.05,       # Flexibility of trend changes
    seasonality_prior_scale=10.0,       # Flexibility of seasonality
    interval_width=0.95,                # Uncertainty interval width
    growth='linear'                     # 'linear' or 'logistic' growth
)

# Fit the model on training data
model.fit(train_data)

print("Model training completed!")

## 8. Make Predictions on Test Data

In [ ]:
# Create future dataframe for the test period
future_test = model.make_future_dataframe(periods=len(test_data), freq='D')

# Make predictions
forecast_test = model.predict(future_test)

# Extract predictions for the test period only
test_predictions = forecast_test.tail(len(test_data))

print("Predictions on test data completed!")
print(f"Test predictions shape: {test_predictions.shape}")

## 9. Model Evaluation

In [ ]:
# Calculate evaluation metrics
actual_prices = test_data['y'].values
predicted_prices = test_predictions['yhat'].values

# Calculate metrics
mae = mean_absolute_error(actual_prices, predicted_prices)
mse = mean_squared_error(actual_prices, predicted_prices)
rmse = np.sqrt(mse)
r2 = r2_score(actual_prices, predicted_prices)

# Calculate percentage errors
mape = np.mean(np.abs((actual_prices - predicted_prices) / actual_prices)) * 100

print("=" * 50)
print("MODEL EVALUATION RESULTS")
print("=" * 50)
print(f"Mean Absolute Error (MAE): ${mae:.2f}")
print(f"Root Mean Square Error (RMSE): ${rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"R-squared Score: {r2:.4f}")
print("=" * 50)

## 10. Visualize Test Results

In [ ]:
# Plot actual vs predicted prices
plt.figure(figsize=(15, 8))

# Plot training data
plt.subplot(2, 1, 1)
plt.plot(train_data['ds'], train_data['y'], label='Training Data', color='blue', alpha=0.7)
plt.plot(test_data['ds'], test_data['y'], label='Actual Test Data', color='green', linewidth=2)
plt.plot(test_data['ds'], predicted_prices, label='Predicted Test Data', color='red', linewidth=2, linestyle='--')
plt.title(f'{STOCK_SYMBOL} Stock Price Prediction - Model Performance', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot residuals
plt.subplot(2, 1, 2)
residuals = actual_prices - predicted_prices
plt.plot(test_data['ds'], residuals, color='purple', linewidth=1)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
plt.title('Prediction Residuals (Actual - Predicted)', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Residual ($)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Future Predictions

In [ ]:
print(f"Making predictions for the next {FORECAST_DAYS} days...")

# Create future dataframe for prediction
future = model.make_future_dataframe(periods=FORECAST_DAYS, freq='D')

# Make predictions
forecast = model.predict(future)

# Extract future predictions only
future_predictions = forecast.tail(FORECAST_DAYS)

print("Future Predictions:")
print("=" * 60)
for i, row in future_predictions.iterrows():
    date = row['ds'].strftime('%Y-%m-%d')
    price = row['yhat']
    lower = row['yhat_lower']
    upper = row['yhat_upper']
    print(f"{date}: ${price:.2f} (${lower:.2f} - ${upper:.2f})")

## 12. Comprehensive Visualization

In [ ]:
# Plot the complete forecast
fig, ax = plt.subplots(figsize=(15, 8))

# Plot historical data
ax.plot(prophet_data['ds'], prophet_data['y'], label='Historical Data', color='black', linewidth=1)

# Plot forecast
forecast_dates = forecast['ds']
ax.plot(forecast_dates, forecast['yhat'], label='Forecast', color='blue', linewidth=2)

# Plot confidence intervals
ax.fill_between(forecast_dates, forecast['yhat_lower'], forecast['yhat_upper'], 
                color='blue', alpha=0.2, label='Confidence Interval')

# Highlight future predictions
future_start = prophet_data['ds'].max()
future_mask = forecast['ds'] > future_start
ax.plot(forecast.loc[future_mask, 'ds'], forecast.loc[future_mask, 'yhat'], 
        color='red', linewidth=3, label='Future Predictions')

# Add vertical line to separate historical and future data
ax.axvline(x=future_start, color='gray', linestyle='--', alpha=0.7, label='Forecast Start')

ax.set_title(f'{STOCK_SYMBOL} Stock Price Forecast with Prophet', fontsize=16)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Price ($)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 13. Prophet Components Analysis

In [ ]:
# Plot forecast components (trend, seasonality, etc.)
fig = model.plot_components(forecast)
plt.suptitle(f'{STOCK_SYMBOL} Forecast Components', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 14. Interactive Prophet Visualization

In [ ]:
# Create interactive plot
fig = plot_plotly(model, forecast)
fig.update_layout(title=f'{STOCK_SYMBOL} Stock Price Forecast - Interactive View')
fig.show()

# Interactive components plot
fig_components = plot_components_plotly(model, forecast)
fig_components.show()

## 15. Model Insights and Summary

In [ ]:
print("=" * 70)
print("PROPHET MODEL ANALYSIS SUMMARY")
print("=" * 70)

# Calculate some insights
total_return = ((prophet_data['y'].iloc[-1] - prophet_data['y'].iloc[0]) / prophet_data['y'].iloc[0]) * 100
current_price = prophet_data['y'].iloc[-1]
predicted_price = future_predictions['yhat'].iloc[-1]
predicted_return = ((predicted_price - current_price) / current_price) * 100

print(f"Stock Symbol: {STOCK_SYMBOL}")
print(f"Analysis Period: {START_DATE} to {END_DATE}")
print(f"Total Historical Return: {total_return:.2f}%")
print(f"Current Price: ${current_price:.2f}")
print(f"Predicted Price ({FORECAST_DAYS} days): ${predicted_price:.2f}")
print(f"Predicted Return: {predicted_return:.2f}%")
print()
print("Model Performance on Test Data:")
print(f"  - Mean Absolute Error: ${mae:.2f}")
print(f"  - Accuracy (100% - MAPE): {100 - mape:.2f}%")
print(f"  - R-squared: {r2:.4f}")
print()
print("Key Insights:")
print("1. Prophet captures trend and seasonality patterns")
print("2. Confidence intervals provide uncertainty estimates")
print("3. Model performance varies with market volatility")
print("4. Use predictions as guidance, not absolute forecasts")
print()
print("⚠️  DISCLAIMER: This is for educational purposes only.")
print("   Stock predictions are inherently uncertain and should")
print("   not be used as the sole basis for investment decisions.")
print("=" * 70)

## 16. Advanced Analysis (Optional)

In [ ]:
# Calculate volatility
returns = prophet_data['y'].pct_change().dropna()
volatility = returns.std() * np.sqrt(252)  # Annualized volatility

# Calculate moving averages for additional context
prophet_data['MA_20'] = prophet_data['y'].rolling(window=20).mean()
prophet_data['MA_50'] = prophet_data['y'].rolling(window=50).mean()

print("Additional Technical Analysis:")
print(f"Annual Volatility: {volatility:.2%}")
print(f"20-day Moving Average: ${prophet_data['MA_20'].iloc[-1]:.2f}")
print(f"50-day Moving Average: ${prophet_data['MA_50'].iloc[-1]:.2f}")

# Plot with moving averages
plt.figure(figsize=(12, 6))
plt.plot(prophet_data['ds'], prophet_data['y'], label='Price', linewidth=1)
plt.plot(prophet_data['ds'], prophet_data['MA_20'], label='20-day MA', alpha=0.7)
plt.plot(prophet_data['ds'], prophet_data['MA_50'], label='50-day MA', alpha=0.7)
plt.title(f'{STOCK_SYMBOL} Price with Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 17. Save Results (Optional)

In [ ]:
# Save predictions to CSV
results_df = pd.DataFrame({
    'Date': future_predictions['ds'],
    'Predicted_Price': future_predictions['yhat'],
    'Lower_Bound': future_predictions['yhat_lower'],
    'Upper_Bound': future_predictions['yhat_upper']
})

filename = f'{STOCK_SYMBOL}_prophet_predictions.csv'
results_df.to_csv(filename, index=False)
print(f"Predictions saved to {filename}")

# Save model evaluation metrics
eval_metrics = {
    'Stock_Symbol': STOCK_SYMBOL,
    'MAE': mae,
    'RMSE': rmse,
    'MAPE': mape,
    'R2_Score': r2,
    'Volatility': volatility,
    'Total_Return': total_return,
    'Predicted_Return': predicted_return
}

eval_df = pd.DataFrame([eval_metrics])
eval_filename = f'{STOCK_SYMBOL}_model_evaluation.csv'
eval_df.to_csv(eval_filename, index=False)
print(f"Model evaluation saved to {eval_filename}")

print("\nAnalysis complete! 🎉")
print("\nTo analyze a different stock:")
print("1. Change the STOCK_SYMBOL variable in Cell 3")
print("2. Adjust START_DATE, END_DATE, and FORECAST_DAYS as needed")
print("3. Run all cells again")

## 18. Deploy Model to AWS SageMaker (Optional)
This section shows how to deploy the trained Prophet model to AWS SageMaker for real-time inference.

In [26]:
# AWS SageMaker Deployment Setup
# First, install required AWS packages
!pip install sagemaker boto3

import boto3
import sagemaker
from sagemaker import get_execution_role
import pickle
import tarfile
import os
from datetime import datetime
import json

print("AWS SageMaker packages installed successfully!")

AWS SageMaker packages installed successfully!


In [27]:
# Configure AWS SageMaker
try:
    # Initialize SageMaker session
    sagemaker_session = sagemaker.Session()
    
    # Get execution role (works in SageMaker Studio/Notebook instances)
    role = get_execution_role()
    
    # Get default S3 bucket
    # bucket = sagemaker_session.default_bucket()
    bucket = "yolo11sagemakerstack-yolo11s31ff79126-3g6ezijge4tc"
    
    print(f"SageMaker Role: {role}")
    print(f"S3 Bucket: {bucket}")
    
except Exception as e:
    print("Note: This cell requires AWS credentials and SageMaker permissions.")
    print("If running locally, configure AWS CLI first: 'aws configure'")
    print(f"Error: {e}")
    
    # For local development, you can manually set these
    # role = 'arn:aws:iam::YOUR-ACCOUNT-ID:role/SageMakerExecutionRole'
    # bucket = 'your-sagemaker-bucket'

SageMaker Role: arn:aws:iam::151182331915:role/YOLO11SageMakerStack-yolo11notebookAccessRole4FE3B3-EdP6dF8ALap0
S3 Bucket: yolo11sagemakerstack-yolo11s31ff79126-3g6ezijge4tc


In [35]:
# Create inference script for SageMaker
inference_code = '''
import os
import pickle
import json
import pandas as pd
from prophet import Prophet
import numpy as np
from datetime import datetime, timedelta

def model_fn(model_dir):
    """Load the Prophet model from the model_dir"""
    model_path = os.path.join(model_dir, 'prophet_model.pkl')
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    return model

def input_fn(request_body, content_type='application/json'):
    """Parse input data for inference"""
    if content_type == 'application/json':
        input_data = json.loads(request_body)
        return input_data
    else:
        raise ValueError(f"Unsupported content type: {content_type}")

def predict_fn(input_data, model):
    """Make predictions using the Prophet model"""
    try:
        # Extract parameters from input
        forecast_days = input_data.get('forecast_days', 30)
        start_date = input_data.get('start_date', None)
        
        # Create future dataframe
        if start_date:
            # If start_date provided, forecast from that date
            future_dates = pd.date_range(start=start_date, periods=forecast_days, freq='D')
            future = pd.DataFrame({'ds': future_dates})
        else:
            # Otherwise, forecast from the end of training data
            future = model.make_future_dataframe(periods=forecast_days, freq='D')
        
        # Make predictions
        forecast = model.predict(future)
        
        # Extract relevant columns and convert to native Python types
        predictions = []
        for _, row in forecast.tail(forecast_days).iterrows():
            predictions.append({
                'date': row['ds'].strftime('%Y-%m-%d'),
                'predicted_price': float(row['yhat']),
                'lower_bound': float(row['yhat_lower']),
                'upper_bound': float(row['yhat_upper']),
                'trend': float(row['trend']) if 'trend' in row else None
            })
        
        return {
            'predictions': predictions,
            'forecast_days': forecast_days,
            'model_info': {
                'model_type': 'Facebook Prophet',
                'prediction_timestamp': datetime.now().isoformat()
            }
        }
        
    except Exception as e:
        return {
            'error': str(e),
            'predictions': []
        }

def output_fn(prediction, accept='application/json'):
    """Format the prediction output"""
    if accept == 'application/json':
        return json.dumps(prediction), 'application/json'
    else:
        raise ValueError(f"Unsupported accept type: {accept}")
'''

print("Inference script created successfully!")

Inference script created successfully!


In [49]:
# Create model artifacts for SageMaker deployment
model_dir = 'prophet_model_artifacts'
code_dir = os.path.join(model_dir, 'code')
os.makedirs(model_dir, exist_ok=True)
os.makedirs(code_dir, exist_ok=True)

# Save the trained Prophet model (in root of model_dir)
model_path = os.path.join(model_dir, 'prophet_model.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

# Save model metadata (in root of model_dir)
metadata = {
    'stock_symbol': STOCK_SYMBOL,
    'training_period': f"{START_DATE} to {END_DATE}",
    'model_performance': {
        'mae': float(mae),
        'rmse': float(rmse),
        'mape': float(mape),
        'r2': float(r2)
    },
    'created_at': datetime.now().isoformat()
}

metadata_path = os.path.join(model_dir, 'metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

# Save inference script (in code/ subdirectory)
inference_script_path = os.path.join(code_dir, 'inference.py')
with open(inference_script_path, 'w') as f:
    f.write(inference_code)

# Create requirements file (in code/ subdirectory)
requirements = '''prophet==1.1.4
pandas>=1.3.0
numpy<2.0
scikit-learn>=1.0.0
torch==1.12
torchvision>=0.9.1'''

requirements_path = os.path.join(code_dir, 'requirements.txt')
with open(requirements_path, 'w') as f:
    f.write(requirements)

print(f"✓ Model saved to: {model_path}")
print(f"✓ Metadata saved to: {metadata_path}")
print(f"✓ Inference script saved to: {inference_script_path}")
print(f"✓ Requirements saved to: {requirements_path}")

# Create model tarball with correct structure
model_tar_path = f'{STOCK_SYMBOL}_prophet_model.tar.gz'

print("Creating tarball with correct structure:")
print("  prophet_model.pkl")
print("  metadata.json")
print("  code/")
print("    ├── inference.py")
print("    └── requirements.txt")

with tarfile.open(model_tar_path, 'w:gz') as tar:
    tar.add(model_dir, arcname='.')

print(f"✓ Model tarball created: {model_tar_path}")
print("Model artifacts ready for SageMaker deployment!")

# Cleanup temporary directory
# import shutil
# shutil.rmtree(model_dir)
# print("✓ Temporary directory cleaned up")

✓ Model saved to: prophet_model_artifacts/prophet_model.pkl
✓ Metadata saved to: prophet_model_artifacts/metadata.json
✓ Inference script saved to: prophet_model_artifacts/code/inference.py
✓ Requirements saved to: prophet_model_artifacts/code/requirements.txt
Creating tarball with correct structure:
  prophet_model.pkl
  metadata.json
  code/
    ├── inference.py
    └── requirements.txt
✓ Model tarball created: AAPL_prophet_model.tar.gz
Model artifacts ready for SageMaker deployment!


In [48]:
# Upload model to S3 and deploy to SageMaker
try:
    # Upload model tarball to S3
    model_s3_uri = sagemaker_session.upload_data(
        path=model_tar_path,
        bucket=bucket,
        key_prefix='prophet-stock-model'
    )
    
    print(f"✓ Model uploaded to S3: {model_s3_uri}")
    
    # Create timestamp for unique naming
    import time
    timestamp = int(time.time())
    model_name = f"prophet-{STOCK_SYMBOL.lower()}-model-{timestamp}"
    endpoint_name = f"prophet-{STOCK_SYMBOL.lower()}-endpoint-{timestamp}"
    
    print(f"Creating SageMaker model: {model_name}")
    print(f"Endpoint name: {endpoint_name}")
    
    # Create SageMaker model using PyTorch container (better for custom inference)
    from sagemaker.pytorch.model import PyTorchModel
    
    prophet_model = PyTorchModel(
        model_data=model_s3_uri,
        role=role,
        entry_point='inference.py',
        framework_version='1.8.1',  # PyTorch version
        py_version='py3',
        name=model_name,
        sagemaker_session=sagemaker_session
    )
    
    print("✓ SageMaker model created successfully!")
    
except Exception as e:
    print(f"Error in model creation: {e}")


✓ Model uploaded to S3: s3://yolo11sagemakerstack-yolo11s31ff79126-3g6ezijge4tc/prophet-stock-model/AAPL_prophet_model.tar.gz
Creating SageMaker model: prophet-aapl-model-1753178379
Endpoint name: prophet-aapl-endpoint-1753178379
✓ SageMaker model created successfully!


In [50]:
# Upload model to S3 and deploy to SageMaker
try:
    # Upload model tarball to S3
    model_s3_uri = sagemaker_session.upload_data(
        path=model_tar_path,
        bucket=bucket,
        key_prefix='prophet-stock-model'
    )
    
    print(f"✓ Model uploaded to S3: {model_s3_uri}")
    
    # Create timestamp for unique naming
    import time
    timestamp = int(time.time())
    model_name = f"prophet-{STOCK_SYMBOL.lower()}-model-{timestamp}"
    endpoint_name = f"prophet-{STOCK_SYMBOL.lower()}-endpoint-{timestamp}"
    
    print(f"Creating SageMaker model: {model_name}")
    print(f"Endpoint name: {endpoint_name}")
    
    # Create SageMaker model using PyTorch container (better for custom inference)
    from sagemaker.pytorch.model import PyTorchModel
    
    prophet_model = PyTorchModel(
        model_data=model_s3_uri,
        role=role,
        entry_point='inference.py',
        framework_version='1.12',  # PyTorch version
        py_version='py38',
        name=model_name,
        sagemaker_session=sagemaker_session
    )
    
    print("✓ SageMaker model created successfully!")
    
except Exception as e:
    print(f"Error in model creation: {e}")


✓ Model uploaded to S3: s3://yolo11sagemakerstack-yolo11s31ff79126-3g6ezijge4tc/prophet-stock-model/AAPL_prophet_model.tar.gz
Creating SageMaker model: prophet-aapl-model-1753178417
Endpoint name: prophet-aapl-endpoint-1753178417
✓ SageMaker model created successfully!


In [51]:
# Deploy model to SageMaker endpoint
try:
    print(f"Deploying model to endpoint: {endpoint_name}")
    print("This may take 5-10 minutes...")
    print("☕ Perfect time for a coffee break!")
    
    # Deploy to endpoint
    predictor = prophet_model.deploy(
        initial_instance_count=1,
        instance_type='ml.t2.medium', 
        endpoint_name=endpoint_name
    )
    
    print("🎉 Model deployed successfully!")
    print(f"Endpoint Name: {endpoint_name}")
    print(f"Endpoint Status: InService")
    
    # Save endpoint info for later use
    endpoint_info = {
        'endpoint_name': endpoint_name,
        'model_name': model_name,
        'stock_symbol': STOCK_SYMBOL,
        'deployment_time': datetime.now().isoformat(),
        'instance_type': 'ml.t2.medium',
        'region': sagemaker_session.boto_region_name
    }
    
    with open(f'{STOCK_SYMBOL}_endpoint_info.json', 'w') as f:
        json.dump(endpoint_info, f, indent=2)
    
    print(f"Endpoint info saved to: {STOCK_SYMBOL}_endpoint_info.json")
    
except Exception as e:
    print(f"Deployment failed: {e}")
    print("Common issues:")
    print("1. Insufficient service limits (check AWS console)")
    print("2. Role permissions (run setup_iam.py script)")
    print("3. Region availability (try us-east-1)")

Deploying model to endpoint: prophet-aapl-endpoint-1753178417
This may take 5-10 minutes...
☕ Perfect time for a coffee break!
---------!🎉 Model deployed successfully!
Endpoint Name: prophet-aapl-endpoint-1753178417
Endpoint Status: InService
Endpoint info saved to: AAPL_endpoint_info.json


In [54]:
# Test the deployed endpoint
try:
    print("Testing the deployed endpoint...")
    
    # Configure predictor to use JSON serialization
    from sagemaker.serializers import JSONSerializer
    from sagemaker.deserializers import JSONDeserializer
    
    predictor.serializer = JSONSerializer()
    predictor.deserializer = JSONDeserializer()
    
    # Test with different scenarios
    test_cases = [
        {'forecast_days': 7, 'description': '7-day forecast'},
        {'forecast_days': 30, 'description': '30-day forecast'},
        {'forecast_days': 1, 'description': 'Next day prediction'}
    ]
    
    for test_case in test_cases:
        print(f"\n--- {test_case['description']} ---")
        
        # Prepare test data
        test_data = {
            'forecast_days': test_case['forecast_days']
        }
        
        try:
            # Make prediction with explicit JSON serialization
            result = predictor.predict(test_data)
            
            if isinstance(result, dict) and 'error' not in result:
                print(f"✓ Success - {len(result.get('predictions', []))} predictions received")
                
                # Show first and last predictions
                predictions = result.get('predictions', [])
                if predictions:
                    first_pred = predictions[0]
                    last_pred = predictions[-1]
                    
                    print(f"First prediction: {first_pred['date']} - ${first_pred['predicted_price']:.2f}")
                    if len(predictions) > 1:
                        print(f"Last prediction:  {last_pred['date']} - ${last_pred['predicted_price']:.2f}")
            else:
                error_msg = result.get('error', 'Unknown error') if isinstance(result, dict) else str(result)
                print(f"✗ Error: {error_msg}")
                
        except Exception as case_error:
            print(f"✗ Test case failed: {case_error}")
    
    print("\n🎉 Endpoint testing completed!")
    print(f"Your endpoint '{endpoint_name}' is ready for use!")
    
except Exception as e:
    print(f"Testing failed: {e}")
    
    # Alternative test using boto3 directly
    print("\nTrying alternative test method using boto3...")
    try:
        import boto3
        import json
        
        runtime = boto3.client('sagemaker-runtime', region_name=sagemaker_session.boto_region_name)
        
        test_payload = {
            'forecast_days': 7
        }
        
        response = runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Body=json.dumps(test_payload)
        )
        
        result = json.loads(response['Body'].read().decode())
        
        if 'predictions' in result:
            print(f"✓ Alternative test successful - {len(result['predictions'])} predictions")
            if result['predictions']:
                first_pred = result['predictions'][0]
                print(f"Sample prediction: {first_pred['date']} - ${first_pred['predicted_price']:.2f}")
        else:
            print(f"✗ Alternative test failed: {result}")
            
    except Exception as alt_error:
        print(f"Alternative test also failed: {alt_error}")
        print("The endpoint might still be starting up. Wait a few minutes and try again.")

Testing the deployed endpoint...

--- 7-day forecast ---
✓ Success - 7 predictions received
First prediction: 2023-03-14 - $150.64
Last prediction:  2023-03-20 - $154.59

--- 30-day forecast ---
✓ Success - 30 predictions received
First prediction: 2023-03-14 - $150.64
Last prediction:  2023-04-12 - $164.62

--- Next day prediction ---
✓ Success - 1 predictions received
First prediction: 2023-03-14 - $150.64

🎉 Endpoint testing completed!
Your endpoint 'prophet-aapl-endpoint-1753178417' is ready for use!
